# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [50]:
# Write your code below.
from dotenv import load_dotenv
load_dotenv()


True

In [51]:
import pandas as pd
import os
import sys
from glob import glob

In [52]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

In [ ]:


# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")



For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [13]:
dd_px.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'Year'],
      dtype='object')

In [31]:
# Write your code below.
dd_feat = dd_px.groupby(dd_px.index).apply(
    lambda x: x.assign(
        Close_lag_1 = x["Close"].shift(1),
        Adj_Close_lag_1 = x["Adj Close"].shift(1),
        returns = (x["Close"] / x["Close"].shift(1)) - 1,
        hi_lo_range = x["High"] - x["Low"]
    ),
)


C:\Users\anria\AppData\Local\Temp\ipykernel_24988\3932964262.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby(dd_px.index).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [29]:
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=89,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [30]:
# Write your code below.

dd_feat.compute()

Date       Open       High        Low      Close  \
ticker ticker                                                          
CWK    CWK    2016-01-27  19.400000  19.879999  19.400000  19.850000   
       CWK    2016-07-26  23.549999  23.860001  23.400000  23.559999   
       CWK    2018-08-02  18.000000  18.180000  17.490000  17.809999   
       CWK    2018-08-03  17.940001  19.450001  17.760000  18.549999   
       CWK    2018-08-06  18.750000  18.790001  18.440001  18.500000   
...                  ...        ...        ...        ...        ...   
QRHC   QRHC   2016-06-27   3.440000   3.440000   2.960000   3.040000   
       QRHC   2016-06-28   3.120000   3.200000   2.880000   3.040000   
       QRHC   2016-06-29   3.120000   3.120000   2.720000   2.720000   
       QRHC   2016-06-30   2.720000   2.800000   1.680000   2.320000   
       QRHC   2016-07-01   2.400000   2.560000   2.400000   2.480000   

               Adj Close      Volume    source  Year  Close_lag_1  \
ticker ticker                                                       
CWK    CWK     19.850000      6417.0   CWK.csv  2016          NaN   
       CWK     23.559999      5855.0   CWK.csv  2016    19.850000   
       CWK     17.809999  14835900.0   CWK.csv  2018    23.559999   
       CWK     18.549999   3003900.0   CWK.csv  2018    17.809999   
       CWK     18.500000   1057700.0   CWK.csv  2018    18.549999   
...                  ...         ...       ...   ...          ...   
QRHC   QRHC     3.040000      4100.0  QRHC.csv  2016     3.360000   
       QRHC     3.040000     85300.0  QRHC.csv  2016     3.040000   
       QRHC     2.720000     25600.0  QRHC.csv  2016     3.040000   
       QRHC     2.320000     66600.0  QRHC.csv  2016     2.720000   
       QRHC     2.480000      6600.0  QRHC.csv  2016     2.320000   

               Adj_Close_lag_1   returns  hi_lo_range  
ticker ticker                                          
CWK    CWK                 NaN       NaN     0.480000  
       CWK           19.850000  0.186902     0.460001  
       CWK           23.559999 -0.244058     0.690001  
       CWK           17.809999  0.041550     1.690001  
       CWK           18.549999 -0.002695     0.350000  
...                        ...       ...          ...  
QRHC   QRHC           3.360000 -0.095238     0.480000  
       QRHC           3.040000  0.000000     0.320000  
       QRHC           3.040000 -0.105263     0.400000  
       QRHC           2.720000 -0.147059     1.120000  
       QRHC           2.320000  0.068966     0.160000  

[328823 rows x 13 columns]

In [39]:
new_feat = dd_feat.compute()

In [45]:
new_feat["returns_10"] = new_feat["returns"].rolling(10).mean()

In [46]:
new_feat.head(10)

Date       Open       High        Low      Close  \
ticker ticker                                                          
CWK    CWK    2016-01-27  19.400000  19.879999  19.400000  19.850000   
       CWK    2016-07-26  23.549999  23.860001  23.400000  23.559999   
       CWK    2018-08-02  18.000000  18.180000  17.490000  17.809999   
       CWK    2018-08-03  17.940001  19.450001  17.760000  18.549999   
       CWK    2018-08-06  18.750000  18.790001  18.440001  18.500000   
       CWK    2018-08-07  18.450001  18.750000  18.100000  18.360001   
       CWK    2018-08-08  18.250000  18.420000  17.799999  18.129999   
       CWK    2018-08-09  18.000000  18.480000  17.889999  18.219999   
       CWK    2018-08-10  18.330000  18.360001  17.990000  18.000000   
       CWK    2018-08-13  18.080000  18.370001  17.750000  17.910000   

               Adj Close      Volume   source  Year  Close_lag_1  \
ticker ticker                                                      
CWK    CWK     19.850000      6417.0  CWK.csv  2016          NaN   
       CWK     23.559999      5855.0  CWK.csv  2016    19.850000   
       CWK     17.809999  14835900.0  CWK.csv  2018    23.559999   
       CWK     18.549999   3003900.0  CWK.csv  2018    17.809999   
       CWK     18.500000   1057700.0  CWK.csv  2018    18.549999   
       CWK     18.360001   1367700.0  CWK.csv  2018    18.500000   
       CWK     18.129999   4004200.0  CWK.csv  2018    18.360001   
       CWK     18.219999    879800.0  CWK.csv  2018    18.129999   
       CWK     18.000000   1808500.0  CWK.csv  2018    18.219999   
       CWK     17.910000    770000.0  CWK.csv  2018    18.000000   

               Adj_Close_lag_1   returns  hi_lo_range  returns_ma_10  \
ticker ticker                                                          
CWK    CWK                 NaN       NaN     0.480000            NaN   
       CWK           19.850000  0.186902     0.460001            NaN   
       CWK           23.559999 -0.244058     0.690001            NaN   
       CWK           17.809999  0.041550     1.690001            NaN   
       CWK           18.549999 -0.002695     0.350000            NaN   
       CWK           18.500000 -0.007568     0.650000            NaN   
       CWK           18.360001 -0.012527     0.620001            NaN   
       CWK           18.129999  0.004964     0.590000            NaN   
       CWK           18.219999 -0.012075     0.370001            NaN   
       CWK           18.000000 -0.005000     0.620001            NaN   

               returns_10  
ticker ticker              
CWK    CWK            NaN  
       CWK            NaN  
       CWK            NaN  
       CWK            NaN  
       CWK            NaN  
       CWK            NaN  
       CWK            NaN  
       CWK            NaN  
       CWK            NaN  
       CWK            NaN

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [ ]:
# Dask is a lazy execution framework: commands will not execute until they are required. Pandas is more convenient to use.
# For large datasets we better use Dask.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.